In [1]:
import glob, json, os, sys

os.environ["LD_LIBRARY_PATH"]="/home/elior/usr/micromamba/envs/swe-bench/lib/"
#sys.path.append('/path/to/metrics/') # TODO: Replace with path to `SWE-bench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

#sys.path.append('/path/to/harness/') # TODO: Replace with path to `SWE-bench/harness` folder
from utils import has_attribute_or_import_error
#sys.path = sys.path[:-1]

/home/elior/usr/micromamba/envs/swe-bench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Declare repository; Fetch tasks, logs

In [2]:
repo = 'pytorch/pytorch' # TODO: Replace with repository name
log_dir = '/home/elior/work/METR/dev/validation_logs' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [3]:
tasks = json.load(open('/home/elior/work/METR/dev/swe-tasks-datasets/versioned_tasks/pytorch-task-instances_versions.json')) # TODO: Replace with path to versioned candidate task instances
tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)
version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

In [4]:
tasks

{'pytorch__pytorch-98055': {'repo': 'pytorch/pytorch',
  'pull_number': 98055,
  'instance_id': 'pytorch__pytorch-98055',
  'issue_numbers': ['97737'],
  'base_commit': 'c263bd43e8e8502d4726643bc6fd046f0130ac0e',
  'patch': 'diff --git a/torch/cuda/amp/autocast_mode.py b/torch/cuda/amp/autocast_mode.py\n--- a/torch/cuda/amp/autocast_mode.py\n+++ b/torch/cuda/amp/autocast_mode.py\n@@ -47,7 +47,7 @@ def _cast(value, dtype):\n     if isinstance(value, torch.Tensor):\n         is_eligible = (value.is_floating_point() and value.is_cuda and (value.dtype is not torch.float64))\n         return value.to(dtype) if is_eligible else value\n-    elif isinstance(value, str):\n+    elif isinstance(value, (str, bytes)):\n         return value\n     elif HAS_NUMPY and isinstance(value, np.ndarray):\n         return value\ndiff --git a/torch/distributed/rendezvous.py b/torch/distributed/rendezvous.py\n--- a/torch/distributed/rendezvous.py\n+++ b/torch/distributed/rendezvous.py\n@@ -90,7 +90,7 @@ def _r

#### Monitor Validation

In [5]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 1
Failed: 0
Usable: 1
Corrupt Test: 0
Corrupt Diff: 0
Test Script Timeout: 0
Success E2E: 1


In [6]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 1
Logs diff: 0


In [12]:
logs_same

['/home/elior/work/METR/dev/validation_logs/pytorch__pytorch-98055.log']

#### Get [FP]2[FP] Tests

In [19]:
inst_id_to_gt = {}
for d in logs_same:
    status_gt = convert_log_to_ground_truth(d,verbose=True)
    inst_id_to_gt[d] = status_gt

[DEBUG] Received log_fp: /home/elior/work/METR/dev/validation_logs/pytorch__pytorch-98055.log
[DEBUG] Save directory: None
[DEBUG] Extracted instance file name: pytorch__pytorch-98055.log
[DEBUG] Extracted repository from log_fp: pytorch/pytorch
[DEBUG] Using log parser for repo 'pytorch/pytorch': <function parse_log_pytest at 0x7e2c9a03fb80>
[DEBUG] Log parsing result: found = True
[DEBUG] Parsed SMS (before/after): [{'test/distributed/test_store.py::FileStoreTest::test_compare_set': 'PASSED', 'test/distributed/test_store.py::FileStoreTest::test_init_pg_and_rpc_with_same_file': 'PASSED', 'test/distributed/test_store.py::FileStoreTest::test_refcount': 'PASSED', 'test/distributed/test_store.py::FileStoreTest::test_set_get': 'PASSED', 'test/distributed/test_store.py::HashStoreTest::test_compare_set': 'PASSED', 'test/distributed/test_store.py::HashStoreTest::test_set_get': 'PASSED', 'test/distributed/test_store.py::PrefixStoreTest::test_get_underlying_store': 'PASSED', 'test/distributed/t

In [20]:
inst_id_to_gt

{'/home/elior/work/METR/dev/validation_logs/pytorch__pytorch-98055.log': {'FAIL_TO_PASS': [],
  'FAIL_TO_FAIL': [],
  'PASS_TO_PASS': ['test/distributed/test_store.py::FileStoreTest::test_compare_set',
   'test/distributed/test_store.py::FileStoreTest::test_init_pg_and_rpc_with_same_file',
   'test/distributed/test_store.py::FileStoreTest::test_refcount',
   'test/distributed/test_store.py::FileStoreTest::test_set_get',
   'test/distributed/test_store.py::HashStoreTest::test_compare_set',
   'test/distributed/test_store.py::HashStoreTest::test_set_get',
   'test/distributed/test_store.py::PrefixStoreTest::test_get_underlying_store',
   'test/distributed/test_store.py::PrefixFileStoreTest::test_compare_set',
   'test/distributed/test_store.py::PrefixFileStoreTest::test_set_get',
   'test/distributed/test_store.py::TCPStoreTest::test_address_already_in_use',
   'test/distributed/test_store.py::TCPStoreTest::test_compare_set',
   'test/distributed/test_store.py::TCPStoreTest::test_init_pg

#### Create Task Instances `.json` file

In [23]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    # if len(v['FAIL_TO_PASS']) == 0:
    #     continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit[task['version']]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{t["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))


Final number of task instances:  1


In [25]:

SAVE_PATH = "/home/elior/work/METR/dev/swe-tasks-datasets/validated_tasks/pytorch__pytorch.json" # TODO: Replace this with a path to a .json file to save the task instances
with open(SAVE_PATH, 'w') as f:
    json.dump(tasks_final, fp=f)